In [2]:
#import git
#git.Git("../").clone("https://github.com/sonos/nlu-benchmark")


In [3]:
import os
import json
import numpy as np
from tensorflow import keras

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-06-08 11:38:08.373220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
DATA_PATH = './nlu-benchmark/2017-06-custom-intent-engines/'
DEFAULT_SLOT_LABEL = 'unspecified'

In [5]:
cls_labels = [ domain for domain in os.listdir(DATA_PATH) if os.path.isdir(DATA_PATH  + domain) ]

In [6]:
cls_train_x = []
cls_train_y = []
cls_val_x = []
cls_val_y = []

Weather_seq_l_train_x = []
Weather_seq_l_train_y = []
Weather_seq_l_val_x = []
Weather_seq_l_val_y = []
weather_seq_label_set = set()

In [7]:
def load_data( domain, dataset_type, full):
    file_path = os.path.join(DATA_PATH, "{}/{}_{}{}.json".format(domain, dataset_type, domain, full))
    with open(file_path, 'r') as filename:
        print(filename)
        dataset = json.load(filename)[domain]
    
    cls_x = []
    cls_y = []
    seq_l_x = []
    seq_l_y = []
    for element in dataset:
        tokenized_sentence = []
        seq_labels = []
        for piece in element['data']:
            words = piece['text'].strip().split()
            tokenized_sentence += words
            for token in words:
                if 'entity' in piece.keys():
                    seq_labels += [piece['entity']]
                else:
                    seq_labels.append(DEFAULT_SLOT_LABEL)
        cls_x.append(tokenized_sentence)
        cls_y.append(domain)
        seq_l_x.append(tokenized_sentence)
        seq_l_y.append(seq_labels)
    return cls_x, cls_y, seq_l_x, seq_l_y
        
        
            
        
    

In [8]:
for domain in cls_labels:
    cls_x, cls_y, seq_l_train_x, seq_l_train_y = load_data(domain, 'train', '_full')
    cls_train_x += cls_x 
    cls_train_y += cls_y
    
    cls_x, cls_y, seq_l_val_x, seq_l_val_y = load_data(domain, 'validate', '')
    cls_val_x += cls_x
    cls_val_y += (cls_y)
    
    if domain == 'GetWeather':
        Weather_seq_l_train_x = seq_l_train_x
        Weather_seq_l_train_y = seq_l_train_y
        Weather_seq_l_val_x = seq_l_val_x
        Weather_seq_l_val_y = seq_l_val_y
        for seq_label in Weather_seq_l_train_y:
            weather_seq_label_set |= set(seq_label)
        for seq_label in Weather_seq_l_val_y:
            weather_seq_label_set |= set(seq_label)            
    
weather_seq_label_list = list(weather_seq_label_set)
weather_seq_label_list.remove('unspecified')
weather_seq_label_list.insert(0, 'unspecified')   

<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/RateBook/train_RateBook_full.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/RateBook/validate_RateBook.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/SearchCreativeWork/train_SearchCreativeWork_full.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/SearchCreativeWork/validate_SearchCreativeWork.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/GetWeather/train_GetWeather_full.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/GetWeather/validate_GetWeather.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/BookRestaurant/train_BookRestaurant_full.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrap

In [9]:
def vectorization(train_x, train_y, valid_x, valid_y, labels, task = 'classification', max_len=64):

    special_tokens = ['<pad>', '<unk>']
    word2ind = {}
    ind2word = {}
    
    def add_word(word2ind, ind2word, word):
        if word in word2ind.keys():
            return
        word2ind[word] = len(ind2word)
        ind2word[len(ind2word)] = word
    for token in special_tokens:
        add_word(word2ind, ind2word, token)
        
    for sent in train_x:
        for word in sent:
            add_word(word2ind, ind2word, word)

    train_x_ids = []
    for sent in train_x:
        sent_ids = []
        for word in sent:
            sent_ids.append(word2ind.get(word, word2ind['<unk>']))
        train_x_ids.append(sent_ids)
        
    train_x_ids = keras.preprocessing.sequence.pad_sequences(train_x_ids, maxlen = max_len, padding = 'post', value = word2ind['<pad>'])
     
    val_x_ids = []
    for sent in valid_x:
        sent_ids = []
        for word in sent:
            sent_ids.append(word2ind.get(word, word2ind['<unk>']))
        val_x_ids.append(sent_ids)
        
    val_x_ids = keras.preprocessing.sequence.pad_sequences(val_x_ids, maxlen = max_len, padding = 'post', value = word2ind['<pad>'])
       
        
        


    if task == 'classification':
        train_y_ids = [labels.index(label) for label in train_y]
        val_y_ids = [labels.index(label) for label in valid_y]
    #classification data vectorization

    elif task == 'slot_labelling':
        train_y_ids =  [[labels.index(label) for label in sent] for sent in train_y]
        train_y_ids = keras.preprocessing.sequence.pad_sequences(train_y_ids, maxlen = max_len, padding = 'post', value = word2ind['<pad>'])
        val_y_ids =  [[labels.index(label) for label in sent] for sent in valid_y]
        val_y_ids = keras.preprocessing.sequence.pad_sequences(val_y_ids, maxlen = max_len, padding = 'post', value = word2ind['<pad>'])
          

    return word2ind, train_x_ids, train_y_ids, val_x_ids,  val_y_ids

In [10]:
cls_word2ind, cls_train_x_ids, cls_train_y_ids, cls_val_x_ids, cls_val_y_ids = vectorization(cls_train_x, cls_train_y,cls_val_x, \
                                                                                             cls_val_y, cls_labels, 'classification' )
Weather_seq_word2ind, Weather_seq_l_train_x_ids, Weather_seq_l_train_y_ids, \
    Weather_seq_l_val_x_ids, Weather_seq_l_val_y_ids =  vectorization(Weather_seq_l_train_x, Weather_seq_l_train_y,\
                                                                      Weather_seq_l_val_x, Weather_seq_l_val_y, weather_seq_label_list, 'slot_labelling')

Weather_seq_sample_weights = np.ones(Weather_seq_l_train_y_ids.shape)
for i, sequence in enumerate(Weather_seq_l_train_y_ids):
    for j, label in enumerate(sequence):
        if label == weather_seq_label_list.index('unspecified'):
            Weather_seq_sample_weights[i][j] = 0.1


In [11]:
import pickle

cls_data = {'word2ind': cls_word2ind,
           'label_list': cls_labels,
           'train_x': cls_train_x_ids,
           'train_y': cls_train_y_ids,
           'val_x': cls_val_x_ids,
           'val_y': cls_val_y_ids}

weather_seq_data = {'word2ind': Weather_seq_word2ind,
           'label_list': weather_seq_label_list,
           'sample_weight': Weather_seq_sample_weights,
           'train_x': Weather_seq_l_train_x_ids,
           'train_y': Weather_seq_l_train_y_ids,
           'val_x': Weather_seq_l_val_x_ids,
           'val_y': Weather_seq_l_val_y_ids}
pickle.dump(cls_data, open('cls_data.pickle', 'wb'))
pickle.dump(weather_seq_data, open('weather_seq_data.pickle', 'wb'))